In [1]:
%matplotlib inline

import pandas as pd
from sklearn.metrics import auc
import matplotlib.pyplot as plt # standard Python plotting library
import numpy as np  # fundamental package for scientific computing, handles arrays and math
import scipy.signal
from scipy.signal import find_peaks, peak_prominences
import scipy.stats as stats
from scipy.stats import zscore
from pathlib import Path
import tdt # import the tdt library
import seaborn as sns
from matplotlib import rcParams
import os
import numpy as np
import glob
from scipy.signal import find_peaks, peak_prominences, peak_widths
import math

In [2]:
isosbestic = '_415A'
dopa='_465A'
CHANNEL = 1
TRANGE       = [-5, 10]
BASELINE_PER = [-5, 2]
event_type = 'RNP_'
het_path = r"N:\2024April_May_Tanks_GRBDA_RI\Day4FR1_04272024\HET/*" # path to het folder remember /*
wt_path = r"N:\2024April_May_Tanks_GRBDA_RI\Day4FR1_04272024\WTY/*"


def T50_measurement(peaks, widths):
    for i in range(len(peaks[0])):
        t50 = (peaks[0][i]-widths[2][i])/(2*101.8) # math for correct time units
    return t50


In [35]:
blockpath = r"n:\2024April_May_Tanks_GRBDA_RI\Day5_RI30_04282024\HET\RI1_EN114_010124-240428-145118"
data = tdt.read_block(blockpath, evtype=['epocs','streams'], channel=CHANNEL)
data = tdt.epoc_filter(data, event_type, t=TRANGE) 

read from t=0s to t=1800.69s


In [51]:
data.epocs.RNP_.onset

array([ 504.51513344,  525.37065472,  654.95793664,  705.8792448 ,
        789.33557248,  990.32236032, 1091.25287936, 1219.44276992,
       1377.95567616, 1424.6928384 , 1727.172608  ])

In [95]:
total_time = (data.info.duration.seconds) + (data.info.duration.microseconds)*1e-6
nbins = 10
interval = total_time/nbins
binned_epocs = []
onsets = data.epocs.RNP_.onset

x = interval
temp_array = []
for i in range(len(onsets)):
    while True:
        if onsets[i] < x:
            temp_array.append(onsets[i])
            break
        elif onsets[i]>x:
            x += interval
            binned_epocs.append(temp_array)
            temp_array = []
            continue
        

a = (onsets[-1])
binned_epocs.append(a)
binned_epocs

[[],
 [],
 [504.51513344, 525.37065472],
 [654.95793664, 705.8792448],
 [789.33557248],
 [990.32236032],
 [1091.25287936, 1219.44276992],
 [1377.95567616, 1424.6928384],
 [],
 1727.172608]

In [87]:
onsets

array([ 504.51513344,  525.37065472,  654.95793664,  705.8792448 ,
        789.33557248,  990.32236032, 1091.25287936, 1219.44276992,
       1377.95567616, 1424.6928384 , 1727.172608  ])

In [ ]:
zall = []
for path in glob.glob(het_path):
    blockpath = path
    data = tdt.read_block(blockpath, evtype=['epocs','streams'], channel=CHANNEL)
    data = tdt.epoc_filter(data, event_type, t=TRANGE)  #filters the chosen epocs (RNP_) and time set around those epocs
    # More examples of list comprehensions
    min1 = np.min([np.size(x) for x in data['streams'][dopa].filtered])
    min2 = np.min([np.size(x) for x in data['streams'][isosbestic].filtered])
    data['streams'][dopa].filtered = [x[1:min1] for x in data['streams'][dopa].filtered]
    data['streams'][isosbestic].filtered = [x[1:min2] for x in data['streams'][isosbestic].filtered]

    # Downsample and average 10x via a moving window mean
    N = 10 # Average every 10 samples into 1 value
    F415 = []
    F465 = []
    for lst in data['streams'][isosbestic].filtered: 
        small_lst = []
        for i in range(0, min2, N):
            small_lst.append(np.mean(lst[i:i+N-1])) # This is the moving window mean
        F415.append(small_lst)

    for lst in data['streams'][dopa].filtered: 
        small_lst = []
        for i in range(0, min1, N):
            small_lst.append(np.mean(lst[i:i+N-1]))
        F465.append(small_lst)

    #Create a mean signal, standard error of signal, and DC offset
    meanF415 = np.mean(F415, axis=0)
    stdF415 = np.std(F415, axis=0)/np.sqrt(len(data['streams'][isosbestic].filtered))
    dcF415 = np.mean(meanF415)
    meanF465 = np.mean(F465, axis=0)
    stdF465 = np.std(F465, axis=0)/np.sqrt(len(data['streams'][dopa].filtered))
    dcF465 = np.mean(meanF465)
    Y_fit_all = []
    Y_dF_all = []
    for x, y in zip(F415, F465):
        x = np.array(x)
        y = np.array(y)
        bls = np.polyfit(x, y, 1)
        fit_line = np.multiply(bls[0], x) + bls[1]
        Y_fit_all.append(fit_line)
        Y_dF_all.append(y-fit_line)
    ts1 = TRANGE[0] + np.linspace(1, len(meanF465), len(meanF465))/data['streams'][dopa].fs*N
    ts2 = TRANGE[0] + np.linspace(1, len(meanF415), len(meanF415))/data['streams'][isosbestic].fs*N
    # Getting the z-score and standard error
    zall = []
    for dF in Y_dF_all: 
        ind = np.where((np.array(ts2)<BASELINE_PER[1]) & (np.array(ts2)>BASELINE_PER[0]))
        zb = np.mean(dF[ind])
        zsd = np.std(dF[ind])
        zall.append((dF - zb)/zsd)
    ts1 = TRANGE[0] + np.linspace(1, len(meanF465), len(meanF465))/data['streams'][dopa].fs*N
    ts2 = TRANGE[0] + np.linspace(1, len(meanF415), len(meanF415))/data['streams'][isosbestic].fs*N
    #ts1 = ts2 since iso and dopamine are recorded simulteanosuly (I can't spell)

    zerror = np.std(zall, axis=0)
